# 7. 代理

<div class="toc">
    <ul class="toc-item">
    <li><span><a href="#7.1-LangChain内置工具" data-toc-modified-id="7.1-LangChain内置工具-1">7.1 LangChain内置工具</a></span>
        <ul class="toc-item">
            <li><span><a href="#7.1.1-使用llm-math和wikipedia工具" data-toc-modified-id="7.1.1-使用llm-math和wikipedia工具-1.1">7.1.1 使用llm-math和wikipedia工具</a></span>
            <li><span><a href="#7.1.2-使用PythonREPLTool工具" data-toc-modified-id="7.1.2-使用PythonREPLTool工具-1.2">7.1.2 使用PythonREPLTool工具</a></span>
            </ul>   
    <li><span><a href="#7.2-定义自己的工具并在代理中使用" data-toc-modified-id="7.2-定义自己的工具并在代理中使用-2">7.2 定义自己的工具并在代理中使用</a></span>
        <ul class="toc-item">
            <li><span><a href="#7.2.1-创建和使用自定义时间工具" data-toc-modified-id="7.2.1-创建和使用自定义时间工具-2.1">7.2.1 创建和使用自定义时间工具</a></span></li>
        </ul>
        </li>
    </ul>
</div>


大语言模型学习并记住许多的网络公开信息，大语言模型最常见的应用场景是，将它当作知识库，让它对给定的问题做出回答。

另一种思路是将大语言模型当作推理引擎，让它基于已有的知识库，并利用新的信息（新的大段文本或者其他信息）来帮助回答问题或者进行推理LongChain的内置代理工具便是适用该场景。

本节我们将会了解什么是代理，如何创建代理， 如何使用代理，以及如何与不同类型的工具集成，例如搜索引擎。

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

## 7.1 LangChain内置工具

In [3]:
# 如果需要查看安装过程日志，可删除 -q 
# -U 安装到最新版本的 wikipedia. 其功能同 --upgrade 
!pip install -U -q wikipedia
!pip install -q openai

In [5]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

### 7.1.1 使用llm-math和wikipedia工具

#### 1️⃣ 初始化大语言模型

- 默认密钥`openai_api_key`为环境变量`OPENAI_API_KEY`。因此在运行以下代码之前，确保你已经设置环境变量`OPENAI_API_KEY`。如果还没有密钥，请[获取你的API Key](https://platform.openai.com/account/api-keys) 。
- 默认模型`model_name`为`gpt-3.5-turbo`。
- 更多关于模型默认参数请查看[这里](https://github.com/hwchase17/langchain/blob/master/langchain/chat_models/openai.py)。

In [24]:
# 参数temperature设置为0.0，从而减少生成答案的随机性。
llm = ChatOpenAI(temperature=0)

#### 2️⃣ 加载工具包
- `llm-math` 工具结合语言模型和计算器用以进行数学计算
- `wikipedia`工具通过API连接到wikipedia进行搜索查询。

In [25]:
tools = load_tools(
    ["llm-math","wikipedia"], 
    llm=llm #第一步初始化的模型
)

#### 3️⃣ 初始化代理

- `agent`: 代理类型。这里使用的是`AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION`。其中`CHAT`代表代理模型为针对对话优化的模型，`REACT`代表针对REACT设计的提示模版。
- `handle_parsing_errors`: 是否处理解析错误。当发生解析错误时，将错误信息返回给大模型，让其进行纠正。
- `verbose`: 是否输出中间步骤结果。

In [26]:
agent= initialize_agent(
    tools, #第二步加载的工具
    llm, #第一步初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)

#### 4️⃣.1️⃣ 使用代理回答数学问题

In [9]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Thought: We need to calculate 25% of 300, which involves multiplication and division.

Action:
```
{
  "action": "Calculator",
  "action_input": "300*0.25"
}
```


Observation: Answer: 75.0
Thought:We have the answer to the question.

Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

✅ **总结**

1. 模型对于接下来需要做什么，给出思考（Thought） 
    
   <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>：我们需要计算300的25%，这个过程中需要用到乘法和除法。</p>

2. 模型基于思考采取行动（Action）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>行动</strong>: 使用计算器（calculator），输入300*0.25</p>
3. 模型得到观察（Observation）
    <p style="font-family:verdana; font-size:12px;color:green"><strong>观察</strong>：答案: 75.0</p>

4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>: 我们的问题有了答案 </p>

5. 给出最终答案（Final Answer）
     <p style="font-family:verdana; font-size:12px;color:green"> <strong>最终答案</strong>: 75.0 </p>
5. 以字典的形式给出最终答案。

#### 4️⃣.2️⃣ Tom M. Mitchell的书

In [29]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
agent(question) 



> Entering new AgentExecutor chain...
Thought: I should use Wikipedia to find the answer to this question.

Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```


Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Int

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Machine Learning.'}

✅ **总结**

1. 模型对于接下来需要做什么，给出思考（Thought） 
   <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>：我应该使用维基百科去搜索。</p>

2. 模型基于思考采取行动（Action）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>行动</strong>: 使用维基百科，输入Tom M. Mitchell</p>
3. 模型得到观察（Observation）
    <p style="font-family:verdana; font-size:12px;color:green"><strong>观测</strong>: 页面: Tom M. Mitchell，页面: Tom Mitchell (澳大利亚足球运动员)</p>

4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>: Tom M. Mitchell写的书是Machine Learning </p>

5. 给出最终答案（Final Answer）
     <p style="font-family:verdana; font-size:12px;color:green"> <strong>最终答案</strong>: Machine Learning </p>
5. 以字典的形式给出最终答案。


值得注意的是，模型每次运行推理的过程可能存在差异，但最终的结果一致。

### 7.1.2 使用PythonREPLTool工具

#### 1️⃣ 创建pyhon代理

In [31]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool

agent = create_python_agent(
    llm,  #使用前面一节已经加载的大语言模型
    tool=PythonREPLTool(), #使用Python交互式环境工具（REPLTool）
    verbose=True #输出中间步骤
)

#### 2️⃣ 使用代理对顾客名字进行排序

In [32]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [33]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
I can use the sorted() function to sort the list of customers by last name and then first name. I will need to provide a key function to sorted() that returns a tuple of the last name and first name in that order.
Action: Python REPL
Action Input:
```
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
for customer in sorted_customers:
    print(customer)
```
Observation: ['Jen', 'Ayai']
['Lang', 'Chain']
['Harrison', 'Chase']
['Elle', 'Elem']
['Trance', 'Former']
['Geoff', 'Fusion']
['Dolly', 'Too']

Thought:The customers are now sorted by last name and then first name. 
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

✅ **总结**

1. 模型对于接下来需要做什么，给出思考（Thought） 
   <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>：我可以使用`sorted()`函数按姓氏和名字的顺序对客户列表进行排序。我需要为`sorted()`提供一个键函数，该函数返回一个元组，元组中按顺序包含姓氏和名字。</p>

2. 模型基于思考采取行动（Action), 因为使用的工具不同，Action的输出也和之前有所不同，这里输出的为python代码
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>行动</strong>: 使用Python编程，输入以下代码</p>
    
    
    ```python
    customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
    sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
    for customer in sorted_customers:
        print(customer)
    ```
    <br>
3. 模型得到观察（Observation）
    <p style="font-family:verdana; font-size:12px;color:green"><strong>观测</strong>:</p>
    
    ```
    ['Jen', 'Ayai']
    ['Lang', 'Chain']
    ['Harrison', 'Chase']
    ['Elle', 'Elem']
    ['Trance', 'Former']
    ['Geoff', 'Fusion']
    ['Dolly', 'Too']
    ```
    <br>
    
4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>: 现在客户按照姓氏和名字的顺序进行了排序。</p>

5. 给出最终答案（Final Answer）
     <p style="font-family:verdana; font-size:12px;color:green"> <strong>最终答案</strong>: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']] </p>
6. 返回最终答案。

#### 3️⃣ 使用调试模式

在调试（debug）模式下再次运行，我们可以把上面的6步分别对应到下面的具体流程
1. 模型对于接下来需要做什么，给出思考（Thought）
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor] Entering Chain run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] [12.25s] Exiting LLM run with output</p>
    - <p style="font-family:verdana; font-size:12px;color:green">[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] [12.25s] Exiting Chain run with output</p>
2. 模型基于思考采取行动（Action), 因为使用的工具不同，Action的输出也和之前有所不同，这里输出的为python代码
    - <p style="font-family:verdana; font-size:12px;color:green"> [tool/start] [1:chain:AgentExecutor > 4:tool:Python REPL] Entering Tool run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [tool/end] [1:chain:AgentExecutor > 4:tool:Python REPL] [2.2239999999999998ms] Exiting Tool run with output</p>
3. 模型得到观察（Observation）   
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor > 5:chain:LLMChain] Entering Chain run with input</p>
4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）   
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/start] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] Entering LLM run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] [6.94s] Exiting LLM run with output</p>
    
5. 给出最终答案（Final Answer） 
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain] [6.94s] Exiting Chain run with output</p>
6. 返回最终答案。
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/end] [1:chain:AgentExecutor] [19.20s] Exiting Chain run with output</p>

In [36]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

## 7.2 定义自己的工具并在代理中使用

In [11]:
# 如果你需要查看安装过程日志，可删除 -q 
!pip install -q DateTime

### 7.2.1 创建和使用自定义时间工具

In [12]:
# 导入tool函数装饰器
from langchain.agents import tool
from datetime import date

#### 1️⃣ 使用tool函数装饰器构建自定义工具
tool函数装饰器可以应用用于任何函数，将函数转化为LongChain工具，使其成为代理可调用的工具。

我们需要给函数加上非常详细的文档字符串, 使得代理知道在什么情况下、如何使用该函数/工具。

比如下面的函数`time`,我们加上了详细的文档字符串

```python
"""
返回今天的日期，用于任何与获取今天日期相关的问题。
输入应该始终是一个空字符串，该函数将始终返回今天的日期。
任何日期的计算应该在此函数之外进行。
"""

```

In [13]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

#### 2️⃣ 初始化代理

In [37]:
agent= initialize_agent(
    tools + [time], #将刚刚创建的时间工具加入到已有的工具中
    llm, #初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)

#### 3️⃣ 使用代理询问今天的日期
**注**: 代理有时候可能会出错（该功能正在开发中）。如果出现错误，请尝试再次运行它。

In [39]:
agent("whats the date today?") 



> Entering new AgentExecutor chain...
Thought: I need to use the `time` tool to get today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2023-06-03
Thought:I have successfully retrieved today's date using the `time` tool.
Final Answer: Today's date is 2023-06-03.

> Finished chain.


{'input': 'whats the date today?', 'output': "Today's date is 2023-06-03."}

✅ **总结**

1. 模型对于接下来需要做什么，给出思考（Thought） 
   <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>：我需要使用 time 工具来获取今天的日期</p>

2. 模型基于思考采取行动（Action), 因为使用的工具不同，Action的输出也和之前有所不同，这里输出的为python代码
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>行动</strong>: 使用time工具，输入为空字符串</p>
    
   
3. 模型得到观察（Observation）
    <p style="font-family:verdana; font-size:12px;color:green"><strong>观测</strong>: 2023-06-03</p>
    
   
    
4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>: 我已成功使用 time 工具检索到了今天的日期</p>

5. 给出最终答案（Final Answer）
     <p style="font-family:verdana; font-size:12px;color:green"> <strong>最终答案</strong>: 今天的日期是2023-06-03.</p>
6. 返回最终答案。